In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Cross Entropy

In [24]:
# logits shape[BS,NC]
batchsize = 2
num_class = 4

#神经网络的输出值
logits = torch.randn(batchsize,num_class) #input unnormalized socre
#目标
target_indices = torch.randint(num_class,size = (batchsize,)) # delta目标分布

In [68]:
## 调用Cross Entropy loss 第一种方法
ce_loss_fn = torch.nn.CrossEntropyLoss()
ce_loss = ce_loss_fn(logits, target_indices)
ce_loss

tensor(1.6490)

In [69]:
## 调用Cross Entropy loss 第二种方法
target_logits = torch.randn(batchsize,num_class)#非delta目标分布
ce_loss = ce_loss_fn(logits, torch.softmax(target_logits,dim=-1))
ce_loss

tensor(1.8511)

In [70]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
input
target = torch.empty(3, dtype=torch.long).random_(5)
target
output = loss(input, target)
output.backward()

In [57]:
input

tensor([[-0.9328,  0.1976, -0.0919,  1.6000,  0.4952],
        [ 0.2369, -1.2858,  1.4548,  0.8806,  0.1214],
        [ 0.6858,  1.6610,  1.6405,  0.7881, -1.0305]], requires_grad=True)

In [58]:
target

tensor([1, 1, 0])

In [59]:
output

tensor(2.5186, grad_fn=<NllLossBackward0>)

In [31]:
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

# 调用Negatice Log Likelihood Loss

In [34]:
nll_fun = torch.nn.NLLLoss()
nll_loss = nll_fun(torch.log(torch.softmax(logits,dim =-1)), target_indices)

In [35]:
nll_loss # 和ce loss 一样 交叉熵就是负的对数似然（所有的样本乘在一起）

tensor(1.6490)

# KL散度

In [42]:
kld_loss_fun = torch.nn.KLDivLoss()

In [44]:
kld_loss = kld_loss_fun(torch.log(torch.softmax(logits,dim =-1)), torch.softmax(target_logits,dim=-1))
kld_loss

tensor(0.1117)

# 验证 交叉熵 = 信息熵 + KL散度

In [78]:
ce_loss_fun_sample = torch.nn.CrossEntropyLoss(reduction="none")
ce_loss_sample = ce_loss_fun_sample(logits, torch.softmax(target_logits,dim=-1))
ce_loss_sample

tensor([1.8164, 1.8859])

In [81]:
kld_loss_fn_sample = torch.nn.KLDivLoss(reduction="none")
kld_loss_sample = kld_loss_fn_sample(torch.log(torch.softmax(logits,dim =-1)), torch.softmax(target_logits,dim=-1))
kld_loss_sample.sum(-1)

tensor([0.7627, 0.8423])

In [86]:
target_information_entropy = torch.distributions.Categorical(probs = torch.softmax(target_logits,dim=-1)).entropy()
target_information_entropy # ie一般为常数 如果目标分布是delt分布 ie = 0

tensor([1.0537, 1.0435])

In [84]:
print(torch.allclose(ce_loss_sample, kld_loss_sample.sum(-1) + target_information_entropy))

True


使用KL loss 和 CELoss 优化是一样的

# 二分类交叉熵

In [99]:
bce_loss_fun = torch.nn.BCELoss()
logits_binary = torch.randn(batchsize)
prob_1 = torch.sigmoid(logits_binary)
target = torch.randint(2,size=(batchsize,))
bce_loss = bce_loss_fun(prob_1,target.float())
bce_loss

tensor(0.7198)